In [1]:
import pandas as pd
train_csv_path = '/home/jupyter-nafisha/normal-abnormal-multitask/CSVs/train.csv'
training_data = pd.read_csv(train_csv_path)

In [2]:
training_data

,image_id,sex,orientation,x_min,y_min,x_max,y_max,class_name,label
0,vinbig/50a418190bc3fb1ef1633bf9678929b3.jpg,M,PA,NaN,NaN,NaN,NaN,No finding,Normal
1,vinbig/051132a778e61a86eb147c7c6f564dfe.jpg,F,PA,1264.0,743.0,1611.0,1019.0,Aortic enlargement,Abnormal
2,vinbig/063319de25ce7edb9b1c6b8881290140.jpg,F,PA,NaN,NaN,NaN,NaN,No finding,Normal
3,vinbig/1c32170b4af4ce1a3030eb8167753b06.jpg,M,PA,627.0,357.0,947.0,433.0,Pleural thickening,Abnormal
4,vinbig/47ed17dcb2cbeec15182ed335a8b5a9e.jpg,F,PA,557.0,2352.0,675.0,2484.0,Nodule/Mass,Abnormal
...,...,...,...,...,...,...,...,...,...
49349,chexpert/patient09201/study1/view2_lateral.jpg,F,lateral,NaN,NaN,NaN,NaN,Lung Opacity,Abnormal
49350,chexpert/patient36078/study1/view1_frontal.jpg,M,AP,NaN,NaN,NaN,NaN,Edema,Abnormal
49351,chexpert/patient40923/study1/view1_frontal.jpg,F,AP,NaN,NaN,NaN,NaN,No Finding,Normal
49352,chexpert/patient02494/study5/view1_frontal.jpg,F,AP,NaN,NaN,NaN,NaN,"Cardiomegaly, Lung Opacity, Pleural Effusion",Abnormal


In [27]:
import os
import json

output_root = "/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/annotations"
records = []                  

for image_id, group in training_data.groupby("image_id"):

    sex = group.iloc[0]["sex"]
    orientation = group.iloc[0]["orientation"]

    # print(image_id, sex, orientation)

    objects = []

    for _, row in group.iterrows():

        x_min, y_min, x_max, y_max = row["x_min"], row["y_min"], row["x_max"], row["y_max"]
        disease_class = row['class_name']

        if any(pd.isna(v) for v in [x_min, y_min, x_max, y_max]):
            continue

        obj = {
            "points": {
                "disease": disease_class,
                "exterior": [[float(x_min), float(y_min)],
                             [float(x_max), float(y_max)]],
                "interior": []
            }
        }

        objects.append(obj)

    data = {
        "sex": sex,
        "orientation": orientation,
        "objects": objects
    }

    # print(data)

    # Maintain folder structure
    json_path = os.path.join(output_root, image_id)
    os.makedirs(os.path.dirname(json_path), exist_ok=True)

    json_path = os.path.splitext(json_path)[0] + ".json"

    with open(json_path, "w") as f:
        json.dump(data, f, indent=4)

    # Store mapping for CSV
    records.append({
        "image_id": image_id,
        "json_path": json_path
    })

In [31]:
# Save CSV mapping
mapping_df = pd.DataFrame(records)
mapping_df.to_csv("/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/CSV/image_json_mapping.csv", index=False)

# print("JSON files and mapping CSV generated successfully.")

In [35]:
mapping_df

,image_id,json_path
0,chexpert/patient00001/study1/view1_frontal.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
1,chexpert/patient00004/study1/view1_frontal.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
2,chexpert/patient00004/study1/view2_lateral.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
3,chexpert/patient00005/study1/view1_frontal.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
4,chexpert/patient00005/study1/view2_lateral.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
...,...,...
33417,vinbig/ff0b6f6423f910a910be0c5729424e58.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
33418,vinbig/ff13acf028a2629a8c4d2bf84c2983cc.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
33419,vinbig/ff4cd09578f32d803e5996a4a2ec29b2.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...
33420,vinbig/ff583c862a0966ce02f1fd7384e4666e.jpg,/home/jupyter-nafisha/normal-abnormal-chexagen...


In [32]:
original_train_data = pd.read_csv('/home/jupyter-nafisha/X-ray-covariates/CSVs/train.csv')

In [52]:
original_train_data

,image_id,sex,class_name,label,orientation
0,chexpert/patient24650/study1/view1_frontal.jpg,M,No Finding,Normal,PA
1,chexpert/patient04301/study2/view1_frontal.jpg,M,No Finding,Normal,PA
2,chexpert/patient49820/study1/view1_frontal.jpg,M,No Finding,Normal,AP
3,chexpert/patient23779/study1/view1_frontal.jpg,M,No Finding,Normal,AP
4,chexpert/patient18206/study1/view1_frontal.jpg,F,No Finding,Normal,PA
...,...,...,...,...,...
33417,chexpert/patient36078/study1/view1_frontal.jpg,M,Edema,Abnormal,AP
33418,chexpert/patient40923/study1/view1_frontal.jpg,F,No Finding,Normal,AP
33419,chexpert/patient02494/study5/view1_frontal.jpg,F,"Cardiomegaly, Lung Opacity, Pleural Effusion",Abnormal,AP
33420,vinbig/9c332e9660d87228aa9d928049230c90.jpg,F,No finding,Normal,PA


In [53]:
len(original_train_data[original_train_data['label']=='Normal']), len(original_train_data[original_train_data['label']=='Abnormal'])

(16202, 17220)

In [54]:
17220+6600

23820

In [36]:
set(original_train_data['image_id'])==set(mapping_df['image_id'])

True

In [37]:
merged_data = original_train_data.merge(
    mapping_df,
    left_on = 'image_id',
    right_on = 'image_id',
    how = 'inner'
)

In [44]:
merged_data.to_csv("/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/CSV/train.csv", index=False)

In [41]:
merged_data['json_path'] = merged_data['json_path'].str.replace('/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/', '')

In [43]:
parent_dir = '/home/jupyter-nafisha/Data/data_v3_CLAHE/'

for _, row in merged_data.iterrows():
    file_path = row['image_id']
    json_path = row['json_path']

    if not os.path.exists(file_path):
        file_path

    if not os.path.exists(json_path):
        json_path

In [49]:
import json
# json_path = '/home/common/TBX11K_v2/tbx11k-DatasetNinja/train/ann/tb1017.png.json'
json_path = '/home/jupyter-nafisha/Data/data_v3_CLAHE/annotations/vinbig/a7ae31b4052444b8f576b657b1753445.json'

with open(json_path, 'r') as f:
    json_data = json.load(f)

print(json_data)

{'sex': 'M', 'orientation': 'PA', 'objects': [{'points': {'disease': 'Pulmonary fibrosis', 'exterior': [[1408.0, 1108.0], [1724.0, 1471.0]], 'interior': []}}, {'points': {'disease': 'Pulmonary fibrosis', 'exterior': [[1452.0, 573.0], [1656.0, 804.0]], 'interior': []}}, {'points': {'disease': 'Pulmonary fibrosis', 'exterior': [[1467.0, 523.0], [1652.0, 808.0]], 'interior': []}}, {'points': {'disease': 'Pulmonary fibrosis', 'exterior': [[1446.0, 529.0], [1659.0, 826.0]], 'interior': []}}, {'points': {'disease': 'Pleural thickening', 'exterior': [[1459.0, 466.0], [1641.0, 587.0]], 'interior': []}}]}


In [51]:
json_data

{'sex': 'M',
 'orientation': 'PA',
 'objects': [{'points': {'disease': 'Pulmonary fibrosis',
    'exterior': [[1408.0, 1108.0], [1724.0, 1471.0]],
    'interior': []}},
  {'points': {'disease': 'Pulmonary fibrosis',
    'exterior': [[1452.0, 573.0], [1656.0, 804.0]],
    'interior': []}},
  {'points': {'disease': 'Pulmonary fibrosis',
    'exterior': [[1467.0, 523.0], [1652.0, 808.0]],
    'interior': []}},
  {'points': {'disease': 'Pulmonary fibrosis',
    'exterior': [[1446.0, 529.0], [1659.0, 826.0]],
    'interior': []}},
  {'points': {'disease': 'Pleural thickening',
    'exterior': [[1459.0, 466.0], [1641.0, 587.0]],
    'interior': []}}]}

# Adding TB data

In [56]:
import os

root = "/home/jupyter-nafisha/Data/data_v3_CLAHE"

merged_data["image_id"] = merged_data["image_id"].apply(
    lambda x: os.path.join(root, x)
)

merged_data["json_path"] = merged_data["json_path"].apply(
    lambda x: os.path.join(root, x)
)

In [57]:
merged_data

,image_id,sex,class_name,label,orientation,json_path
0,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
1,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
2,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
3,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
4,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,F,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
...,...,...,...,...,...,...
33417,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,Edema,Abnormal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
33418,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,F,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
33419,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,F,"Cardiomegaly, Lung Opacity, Pleural Effusion",Abnormal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
33420,/home/jupyter-nafisha/Data/data_v3_CLAHE/vinbi...,F,No finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...


In [59]:
for _, row in merged_data.iterrows():
    file_path = row['image_id']
    json_path = row['json_path']

    if not os.path.exists(file_path):
        file_path

    if not os.path.exists(json_path):
        json_path

    break

In [67]:
additional_data= pd.read_csv('../normal-abnormal-multitask/CSVs/trainWithTB-withAdditionalNormal.csv')
TB_data= additional_data[additional_data['image_id'].str.startswith('/home/common')]

In [69]:
import numpy as np
len(np.unique(TB_data['image_id']))

473

In [72]:
TB_data= TB_data[['image_id', 'sex', 'class_name', 'label', 'orientation']]

In [81]:
list(TB_data['image_id'])[0]

'/home/common/TBX11K_v2/tbx11k-DatasetNinja/train/img/tb0662.png'

In [83]:
import os

TB_data["json_path"] = TB_data["image_id"].apply(
    lambda x: x + ".json"
)

/tmp/ipykernel_15403/7323416.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TB_data["json_path"] = TB_data["image_id"].apply(


In [95]:
TB_data['json_path'] = TB_data['json_path'].str.replace('img', 'ann')

/tmp/ipykernel_15403/935961633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TB_data['json_path'] = TB_data['json_path'].str.replace('img', 'ann')


In [85]:
TB_data = TB_data.drop_duplicates(subset=["image_id"])

In [96]:
for _, row in TB_data.iterrows():
    file_path = row['image_id']
    json_path = row['json_path']

    if not os.path.exists(file_path):
        file_path

    if not os.path.exists(json_path):
        json_path

    # break

In [97]:
os.path.exists(json_path)

True

In [98]:
json_path

'/home/common/TBX11K_v2/tbx11k-DatasetNinja/train/ann/tb0200.png.json'

In [94]:
os.path.exists('/home/common/TBX11K_v2/tbx11k-DatasetNinja/train/img')

True

In [103]:
merged_data.columns, TB_data.columns

(Index(['image_id', 'sex', 'class_name', 'label', 'orientation', 'json_path'], dtype='object'),
 Index(['image_id', 'sex', 'class_name', 'label', 'orientation', 'json_path'], dtype='object'))

In [106]:
merged_data_2 = pd.concat([merged_data, TB_data], ignore_index=True)

In [107]:
merged_data_2

,image_id,sex,class_name,label,orientation,json_path
0,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
1,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
2,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
3,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
4,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,F,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
...,...,...,...,...,...,...
33890,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...,NaN,right_active_tb,Abnormal,NaN,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...
33891,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...,NaN,right_active_tb,Abnormal,NaN,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...
33892,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...,NaN,left_active_tb,Abnormal,NaN,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...
33893,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...,NaN,right_active_tb,Abnormal,NaN,/home/common/TBX11K_v2/tbx11k-DatasetNinja/tra...


In [108]:
len(merged_data_2[merged_data_2['label']=='Normal']), len(merged_data_2[merged_data_2['label']=='Abnormal'])

(16202, 17693)

In [109]:
merged_data

,image_id,sex,class_name,label,orientation,json_path
0,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
1,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
2,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
3,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
4,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,F,No Finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
...,...,...,...,...,...,...
33417,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,M,Edema,Abnormal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
33418,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,F,No Finding,Normal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
33419,/home/jupyter-nafisha/Data/data_v3_CLAHE/chexp...,F,"Cardiomegaly, Lung Opacity, Pleural Effusion",Abnormal,AP,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...
33420,/home/jupyter-nafisha/Data/data_v3_CLAHE/vinbi...,F,No finding,Normal,PA,/home/jupyter-nafisha/Data/data_v3_CLAHE/annot...


In [110]:
merged_data_2.to_csv("/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/CSV/train.csv", index=False)

### csv

In [150]:
training_data = pd.read_csv('/home/jupyter-nafisha/X-ray-covariates/CSVs/train.csv')

In [151]:
training_data = training_data[training_data['label']=='Abnormal']
training_data = training_data[training_data['image_id'].str.startswith('vinbig/')]

In [152]:
training_data

,image_id,sex,class_name,label,orientation
5,vinbig/a7ae31b4052444b8f576b657b1753445.jpg,M,"Pleural thickening, Pulmonary fibrosis",Abnormal,PA
6,vinbig/d0043062cebce85f2487407bc033d405.jpg,M,"Aortic enlargement, Atelectasis, Cardiomegaly,...",Abnormal,PA
18,vinbig/52951d7de2485aba8ed62629eee4d254.jpg,F,"Cardiomegaly, Other lesion",Abnormal,PA
40,vinbig/828468076a2fa7c97169cfb83a117593.jpg,M,"Infiltration, Pulmonary fibrosis",Abnormal,PA
47,vinbig/18d37c314615dede5fb25722c8b2ee04.jpg,F,Cardiomegaly,Abnormal,PA
...,...,...,...,...,...
33394,vinbig/a30e54c94c56cf1448eb8f3921a40e15.jpg,M,"Aortic enlargement, Cardiomegaly",Abnormal,PA
33395,vinbig/b5e2ebedcb69b842ee6b751e5a4991a8.jpg,M,"Aortic enlargement, Cardiomegaly",Abnormal,PA
33398,vinbig/765760a9e2d3b82da72ca1ab86229b79.jpg,F,"Aortic enlargement, Cardiomegaly, Consolidatio...",Abnormal,PA
33399,vinbig/466b5b5f21f766aecb22148002ff865a.jpg,F,Aortic enlargement,Abnormal,PA


In [129]:
val_data.to_csv("/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/CSV/val.csv", index=False)

In [153]:
2240 + 473

2713

In [134]:
import pandas as pd

csv_path = "/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/CSV/val.csv"

df = pd.read_csv(csv_path)

# Split by label
normal_df = df[df["label"] == "Normal"]
abnormal_df = df[df["label"] == "Abnormal"]

# Sample safely
normal_sample = normal_df.sample(n=50, random_state=42)
abnormal_sample = abnormal_df.sample(n=50, random_state=42)

# Combine
balanced_df = pd.concat([normal_sample, abnormal_sample], ignore_index=True)

# Optional shuffle
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save
balanced_df.to_csv("/home/jupyter-nafisha/normal-abnormal-chexagent-attention-loss/CSV/val_subset_100.csv", index=False)

In [135]:
len(balanced_df[balanced_df['label']=='Abnormal']), len(balanced_df[balanced_df['label']=='Normal'])

(50, 50)